In [1]:
from fomo.models.clip.clip_base import ClipBase
import torch
from PIL import Image

/Users/dominykas.seputis/github/fomo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# CLIP Base Demo

In [2]:
clip = ClipBase()
clip.to_cpu()
clip.eval();

In [3]:
clip.precompute_prompt_features(["a picture of a cat", "a picture of kitten"])

In [4]:
img = Image.open("./cat.jpg")

In [5]:
img_tensor = clip.transform(img).view(1, 3, 224, 224)

In [7]:
with torch.no_grad():
    logits = clip.forward(img_tensor)
print(logits)

tensor([[29.0192, 26.0525]])


In [8]:
torch.functional.F.softmax(logits, dim=1)

tensor([[0.9510, 0.0490]])

# Datasets

In [3]:
from fomo.utils.data.datasets import DatasetInitializer
from fomo.utils.data import utils

In [4]:
# loading zero_shot_dataset
zero_shot_dataset = DatasetInitializer.from_str("cifar10").value(train=True)

# zero_shot_dataset contains two properties: torch dataset and labels
print(zero_shot_dataset.dataset)
print(zero_shot_dataset.labels)

Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [5]:
# you can also split the dataset into train and eval splits using utils

# using percentage split
train_dataset, val_dataset = utils.split_train_val(zero_shot_dataset.dataset, train_size=0.8)
print(len(train_dataset), len(val_dataset))

40000 10000


In [6]:
# using number of samples split

train_dataset, val_dataset = utils.split_train_val(zero_shot_dataset.dataset, train_eval_samples=[10, 20])
print(len(train_dataset), len(val_dataset))

10 20


In [11]:
# using subsample split
subsampled_dataset = utils.subsample_classes(zero_shot_dataset.dataset, "all")
print(subsampled_dataset.__len__())

subsampled_dataset = utils.subsample_classes(zero_shot_dataset.dataset, "base")
print(subsampled_dataset.__len__())

subsampled_dataset = utils.subsample_classes(zero_shot_dataset.dataset, "new")
print(subsampled_dataset.__len__())

50000
25000
25000


# Learner

In [1]:
from fomo.pipelines.train import Learner
from fomo.pipelines.types.learner_args import LearnerArgs

/Users/dominykas.seputis/github/fomo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
learner_args = LearnerArgs()
learner_args.device = "cpu"
learner_args.epochs = 1
learner_args.model_type = "clip_linear"
learner_args.use_wandb = True
learner_args.train_subsample = "base"
learner_args.test_subsample = "new"
learner_args.train_eval_size = (10, 10)

learner = Learner(learner_args)

Files already downloaded and verified
Files already downloaded and verified
Turning off gradients in both the image and the text encoder
Parameters to be updated: {'image_linear.0.weight', 'image_linear.0.bias'}
Number of learnable paramms: 262656


/Users/dominykas.seputis/github/fomo/.venv/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [3]:
learner.run()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dqmiss. Use `wandb login --relogin` to force relogin


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: [0][0/1]	Time 11.054 (11.054)	Data 10.446 (10.446)	Loss 2.6379e+00 (2.6379e+00)	Acc@1   0.00 (  0.00)


100%|██████████| 1/1 [00:10<00:00, 10.45s/it]

Validate: [0/1]	Time 10.451 (10.451)	Loss 1.8925e+00 (1.8925e+00)	Prompt Acc@1  20.00 ( 20.00)


...

Validate: [60/79]	Time  3.536 ( 3.518)	Loss 1.6810e+00 (1.6547e+00)	Prompt Acc@1  18.75 ( 24.59)


 90%|████████▉ | 71/79 [04:08<00:27,  3.39s/it]

Validate: [70/79]	Time  3.356 ( 3.506)	Loss 1.5504e+00 (1.6482e+00)	Prompt Acc@1  32.81 ( 25.13)


100%|██████████| 79/79 [04:53<00:00,  3.71s/it]

 * Prompt Acc@1 24.960


## Extending ClipBase

In [10]:
# import ClipBase
from fomo.models.clip.clip_base import ClipBase
from torch import nn
import torch

In [13]:
class ClipExtension(ClipBase):
    def __init__(self, backbone: str = "ViT-B/16", root: str = "./data") -> None:
        # pass default arguments to the parent class
        super(ClipExtension, self).__init__(backbone, root=root)

        # add additional blocks to the model

        self.visual_mlp = nn.Sequential(
            nn.Linear(self._clip.visual.output_dim, 12),
            nn.Linear(12, self._clip.visual.output_dim)
        )

    @property
    def learnable_param_names(self) -> set[str]:
         # IMPORTANT: Add the name of the learnable parameters in the model
        return set(["image_linear"])

    # If needed you can override the to_cpu and to_cuda methods
    def to_cpu(self) -> None:
        self._clip.to(torch.device("cpu"))
        self.image_linear.to(torch.device("cpu"))
        self._clip.float()

    def to_cuda(self) -> None:
        self.image_linear.to(torch.device("cuda"))
        self._clip.to(torch.device("cuda"))

    def forward(self, images: torch.Tensor, prompts: list[str] | None = None) -> torch.Tensor:
        # Change the forward method to include the visual_mlp
        if prompts:
            text_features = self.encode_text(prompts)
        elif self._precomputed_prompt_features is not None:
            text_features = self._precomputed_prompt_features
        else:
            raise ValueError("At least one prompts or pre-computed promt features has to be present.")

        image_features = self.encode_images(images)

        image_features = self.image_linear(image_features)

        logits_per_image: torch.Tensor = self.logit_scale * image_features @ text_features.t()

        return logits_per_image

In [15]:
model = ClipExtension()

#print learnable parameters
print(model.learnable_param_names)

{'image_linear'}
